In [1]:
import sys
sys.path.append('/NDDepth/src')

In [2]:
from PIL import Image
import torch
import tqdm
from torch.utils.data import DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
from model import Model, ModelConfig
from dataloader import ImageDataset, preprocess_transform
from loss import silog_loss, histogram_intersection_loss

torch.manual_seed(42)

In [3]:
train_dataset = ImageDataset('/scratchdata/nyu_data', '/scratchdata/nyu_data/data/nyu2_train.csv', transform=preprocess_transform)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [4]:
config =  ModelConfig("tiny07")
model = Model(config).to("cuda")

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [6]:

for i, x in enumerate(tqdm.tqdm(train_dataloader)):
    
    for k in x.keys():
        x[k] = x[k].to("cuda")
        
    d1, d2 = model(x)
    
    gt = x["depth_values"]
    pred1 = F.interpolate(d1[-1], size=gt.shape[2:], mode='bilinear', align_corners=False)
    pred2 = F.interpolate(d2[-1], size=gt.shape[2:], mode='bilinear', align_corners=False)
    
    loss = silog_loss(pred1, gt) + silog_loss(pred2, gt) * 10

    break

  0%|          | 0/50688 [00:00<?, ?it/s]


In [7]:
print(loss)

tensor(68.3375, device='cuda:0', grad_fn=<AddBackward0>)
